In [1]:
import sqlite3
conn = sqlite3.connect("AQI.db")
sql = '''
CREATE TABLE IF NOT EXISTS records (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	sitename TEXT NOT NULL,
	county TEXT,
	aqi INTEGER,
	status TEXT,
	pm25 NUMERIC,
	date TEXT,
	lat NUMERIC,
	lon NUMERIC,
    UNIQUE(sitename,date)
);
'''
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
cursor.close()
conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect("AQI.db")
# 以下為 context manager 寫法，不須close, commit（但後者仍建議寫出來）
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [4]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['嘉義', '恆春', '沙鹿', '永和', '安南', '冬山', '平鎮', '臺南（麻豆）', '基隆', '麥寮', '三義', '淡水', '汐止', '鳳山', '臺東', '忠明', '三重', '萬華', '大里', '金門', '龍潭', '士林', '桃園', '臺西', '關山', '板橋', '左營', '復興', '土城', '南投', '頭份', '新莊', '彰化', '屏東(枋山)', '新竹', '崙背', '新港', '楠梓', '中壢', '大園', '仁武', '二林', '新店', '高雄（湖內）', '西屯', '彰化（員林）', '萬里', '富貴角', '湖口', '竹東', '陽明', '林口', '古亭', '中山', '花蓮', '林園', '屏東', '松山', '屏東（琉球）', '新營', '美濃', '宜蘭（頭城）', '馬祖', '前金', '潮州', '苗栗', '觀音', '朴子', '橋頭', '小港', '大城', '馬公', '豐原', '大同', '竹山', '大寮', '新北(樹林)', '線西', '菜寮', '宜蘭', '臺南', '埔里', '善化', '前鎮', '斗六']


In [2]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 06:00', '屏東縣', '35', '4', '良好', '22.260899', '120.651472'],
 ['2024-11-04 06:00', '屏東縣', '35', '4', '良好', '22.260899', '120.651472'],
 ['2024-11-04 05:00', '屏東縣', '35', '3', '良好', '22.260899', '120.651472'],
 ['2024-11-04 05:00', '屏東縣', '35', '3', '良好', '22.260899', '120.651472']]

In [3]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [5]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?); '''
            #上面第一句指定DB中要插入的"欄位"；第二句表明要替換的數值
            #第一句如果不明確指定要插入的欄位名稱，則默認情況下，SQL 認為插入值必須與DB表中的所有欄位數量和順序完全匹配。但DB中若有其他欄位如id，那就會出問題
            #這兩句為SQL語法，處理DB內部的工作；第一句先定義了值的擺放欄位；第二句則是處理了值的傳入
            
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon)) 
            #上句是Python語法，連結Python與SQL。tuple會把填入的各python變數，傳送並指定給sql語句中values中的替位符"?"，然後再傳送並擺放給第一句中所指定的欄位


In [ ]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)